In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import Libaries

In [4]:
#Tokenizer Vietnamese
!pip install pyvi

In [5]:
import pandas as pd
import string
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from pyvi import ViTokenizer
from sklearn.model_selection import train_test_split
import seaborn as sb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import  LogisticRegression
from sklearn import svm
from sklearn.metrics import confusion_matrix, f1_score, classification_report, accuracy_score, precision_score, recall_score

#Data Loading


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
dataset = pd.read_csv('/content/drive/MyDrive/Datacs114 - Sheet1.csv')
dataset

,Công an bắt tạm giam Nguyễn Minh Phúc,1
0,Quảng Bình: Phó bí thư Đảng ủy xã bị bắt quả t...,1.0
1,Vụ nữ sinh lớp 12 tử vong: Gia đình không khán...,1.0
2,Trang Nemo đã thi hành án 9 tháng tù?,1.0
3,Quảng Ninh: Độc quyền của chủ mỏ gây khó khăn ...,1.0
4,TP.HCM: Bất ngờ lời khai của nghi phạm cướp gi...,1.0
...,...,...
5101,Vợ tạt nước sôi chồng vì ghen,1.0
5102,Người chồng tông xe máy hại chết vợ trong cơn ...,1.0
5103,Gã đàn ông ở Bà Rịa - Vũng Tàu đoạt mạng người...,1.0
5104,Chồng phi tang xác vợ dưới giếng trong cơn ghen,1.0


In [9]:
dataset.shape

(5106, 2)

#Data Processing

In [10]:
dataset.columns = ['Title','Is_neg']
dataset.head()

,Title,Is_neg
0,Quảng Bình: Phó bí thư Đảng ủy xã bị bắt quả t...,1.0
1,Vụ nữ sinh lớp 12 tử vong: Gia đình không khán...,1.0
2,Trang Nemo đã thi hành án 9 tháng tù?,1.0
3,Quảng Ninh: Độc quyền của chủ mỏ gây khó khăn ...,1.0
4,TP.HCM: Bất ngờ lời khai của nghi phạm cướp gi...,1.0


In [11]:
#Remove NaN
if dataset['Title'].isnull().values.any():
  dataset = dataset.dropna(subset=['Title'])
dataset.shape

(5067, 2)

In [12]:
#Remove Duplication
if dataset.duplicated('Title').any():
  dataset = dataset.drop_duplicates(subset='Title')
dataset.shape

(4518, 2)

In [13]:
#Lowercasing
dataset['Title'] = dataset['Title'].str.lower()
dataset.head()

,Title,Is_neg
0,quảng bình: phó bí thư đảng ủy xã bị bắt quả t...,1.0
1,vụ nữ sinh lớp 12 tử vong: gia đình không khán...,1.0
2,trang nemo đã thi hành án 9 tháng tù?,1.0
3,quảng ninh: độc quyền của chủ mỏ gây khó khăn ...,1.0
4,tp.hcm: bất ngờ lời khai của nghi phạm cướp gi...,1.0


In [14]:
#Remove Punctuations
def remove_punctuation(title):
    title = [char for char in title if char not in string.punctuation]
    removed = ''.join(title)
    return removed
dataset['Title'] = dataset['Title'].apply(remove_punctuation)
dataset.head()

,Title,Is_neg
0,quảng bình phó bí thư đảng ủy xã bị bắt quả ta...,1.0
1,vụ nữ sinh lớp 12 tử vong gia đình không kháng...,1.0
2,trang nemo đã thi hành án 9 tháng tù,1.0
3,quảng ninh độc quyền của chủ mỏ gây khó khăn c...,1.0
4,tphcm bất ngờ lời khai của nghi phạm cướp giật...,1.0


In [15]:
#Tokenizer
dataset['Title'] = dataset['Title'].apply(ViTokenizer.tokenize)
dataset.head()

,Title,Is_neg
0,quảng_bình phó bí_thư đảng_ủy xã bị bắt quả_ta...,1.0
1,vụ nữ_sinh lớp 12 tử_vong gia_đình không kháng...,1.0
2,trang nemo đã thi_hành án 9 tháng tù,1.0
3,quảng_ninh độc_quyền của chủ mỏ gây khó_khăn c...,1.0
4,tphcm bất_ngờ lời khai của nghi phạm cướp_giật...,1.0


In [16]:
#Remove stopwords
import requests

# URL of file stopwords
url = 'https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords.txt'

# Request
response = requests.get(url)

if response.status_code == 200:
    # Save file 'vietnamese-stopwords.txt'
    with open('vietnamese-stopwords.txt', 'w') as f:
        f.write(response.text)
else:
    print('Error:', response.status_code)

with open('vietnamese-stopwords.txt', 'r') as f:
    vietnamese_stopwords = f.read().splitlines()

In [17]:
vietnamese_stopwords

['a lô',
 'a ha',
 'ai',
 'ai ai',
 'ai nấy',
 'ai đó',
 'alô',
 'amen',
 'anh',
 'anh ấy',
 'ba',
 'ba ba',
 'ba bản',
 'ba cùng',
 'ba họ',
 'ba ngày',
 'ba ngôi',
 'ba tăng',
 'bao giờ',
 'bao lâu',
 'bao nhiêu',
 'bao nả',
 'bay biến',
 'biết',
 'biết bao',
 'biết bao nhiêu',
 'biết chắc',
 'biết chừng nào',
 'biết mình',
 'biết mấy',
 'biết thế',
 'biết trước',
 'biết việc',
 'biết đâu',
 'biết đâu chừng',
 'biết đâu đấy',
 'biết được',
 'buổi',
 'buổi làm',
 'buổi mới',
 'buổi ngày',
 'buổi sớm',
 'bà',
 'bà ấy',
 'bài',
 'bài bác',
 'bài bỏ',
 'bài cái',
 'bác',
 'bán',
 'bán cấp',
 'bán dạ',
 'bán thế',
 'bây bẩy',
 'bây chừ',
 'bây giờ',
 'bây nhiêu',
 'bèn',
 'béng',
 'bên',
 'bên bị',
 'bên có',
 'bên cạnh',
 'bông',
 'bước',
 'bước khỏi',
 'bước tới',
 'bước đi',
 'bạn',
 'bản',
 'bản bộ',
 'bản riêng',
 'bản thân',
 'bản ý',
 'bất chợt',
 'bất cứ',
 'bất giác',
 'bất kì',
 'bất kể',
 'bất kỳ',
 'bất luận',
 'bất ngờ',
 'bất nhược',
 'bất quá',
 'bất quá chỉ',
 'bất thình l

In [18]:
dataset['Title'] = dataset['Title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (vietnamese_stopwords)]))
dataset.head()

,Title,Is_neg
0,quảng_bình phó bí_thư đảng_ủy xã bắt quả_tang ...,1.0
1,vụ nữ_sinh lớp 12 tử_vong gia_đình kháng_cáo,1.0
2,trang nemo thi_hành án 9 tù,1.0
3,quảng_ninh độc_quyền chủ mỏ khó_khăn san_lấp d...,1.0
4,tphcm bất_ngờ khai nghi phạm cướp_giật thủ đức,1.0
